# Die Webseite der TH Nürnberg- Intranet Scrapen

### Einleitung

Die Webseite der TH-Nürnberg wirkt als Ausgangspunkt für die Wissensgrundlage des Chatbots.

Conventions: 
- pandas Spaltennamen im Sigular
- Die meißten Links sind keine URLs, da sie lokal sind

In [3]:
import sys
sys.path.append('..')
from bs4 import BeautifulSoup
from bs4.element import Comment
import pandas as pd
from db_init import db_get_df, db_save_df
import glob
import json
import requests
import sys
from tqdm import tqdm

**Note:** Um diesen Notebook auszuführen, braucht man eine VPN Verbindung zur TH-Intranet.

### Scrapen der home Seite

Als Ausganspunkt für unsere Daten nutzen wir die Home Webseite des TH-Nürnberg-Intranet. (https://intern.ohmportal.de/)
Diese Website downloaden wir und suchen alle Links auf andere Webseiten und speichern diese Links in eine Liste.
Als nächsten Schritt rufen wir alle Links aus dieser Liste auf und sammeln wiederum alle Links von jeder dieser Webseiten.
In der daraus resultierenden Liste sortieren wir alle Links aus, die nicht auf die Webseite des TH-Intranet verweisen.
Dann laden wir alle Dokumente herunter und speichern sie in der Datenbank.

Eine Funktion, die eine URL als parameter nimmt und das HTML file zurückgibt, falls die Seite existiert.

Dafür nutzen wir die requests Bibliothek.

Da einige Seiten eine Umleitung auf die THN Webseite enthalten, überprüfen wir, ob die URL nach dem Umleiten noch zur ursprünglichen Domäne gehört.

In [88]:
def download_html_from_url(url):
    res = requests.get(url)
    html = ""
    
    if res.status_code == 200:
        if res.url.startswith(url):
            html = res.text
        else:
            print(f"Umleitung zu externer Seite verhindert. URL: {res.url}")
    else:
        print(f"Kein Inhalt heruntergeladen. Statuscode: {res.status_code}")
    
    return html


Eine Funktion, die ein HTML file nach Links durchsucht und alle gefundenen externen und internen Links zurückgibt.
Dafür nutzen wir die Bibliothek Beautifulsoup, mit dem lxml parser.

In [93]:
def get_links_from_one_html(html):
    soup = BeautifulSoup(html,"lxml")
    links = [a["href"] for a in soup.find_all('a', href=True)]
    return links

Jetzt geben wir unsere initiale URL an und extrahieren alle Links aus dieser

In [196]:
BASE_URL = "https://intern.ohmportal.de/"
html = download_html_from_url(BASE_URL)
links = get_links_from_one_html(html)
df = pd.DataFrame({"link": links})
print(*df["link"])
print(len(df["link"]))

/seitenbaum/home/page.html /seitenbaum/hochschule/page.html /seitenbaum/fakultaeten/page.html /seitenbaum/kontakt/page.html /seitenbaum/impressum/page.html /seitenbaum/datenschutz/page.html /seitenbaum/hilfe-in-notfaellen/page.html /seitenbaum/studierende/fuer-studierende/page.html /seitenbaum/lehrende-und-beschaeftigte/fuer-lehrende-und-beschaeftigte/page.html /seitenbaum/home/page.html /seitenbaum/home/allgemeinwissenschaftliche-wahl-wahlpflichtfaecher/page.html /seitenbaum/home/bibliothek/page.html /seitenbaum/home/career-service/page.html /seitenbaum/home/contacting/page.html /seitenbaum/home/hochschulkommunikation/page.html /seitenbaum/home/hochschulleitung/page.html /institutionen/hochschulservice-fuer-familie-gleichstellung-und-gesundheit/page.html /seitenbaum/hochschule/oeffnungszeiten-der-hochschule/page.html /seitenbaum/home/qualitaetsmanagement/page.html /seitenbaum/home/lehr-und-kompetenzentwicklung/page.html /institutionen/zentrale-studienberatung/page.html /seitenbaum/hom

c:\Users\lizab\projects\Daibl\daibl\.venv\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


### Links filtern

Zunächst können wir alle Links überprüfen, ob sie Parameter oder sections mit 
- *?param1=hallo*
- *#section* 

enthalten. Beide Attribute sind für den Download der Webseiten nicht notwendig und werden ausgefiltert. Dies spart uns HTML Duplikate.

In [95]:
def clean_links(links):
    cleaned_links = [link.split('#')[0].split('?')[0] for link in links]
    return cleaned_links

Nun speichern wir das zwischen Ergebnis der gefilterten Links.

In [197]:
df["link"]=clean_links(df["link"])
print(*df["link"])


/seitenbaum/home/page.html /seitenbaum/hochschule/page.html /seitenbaum/fakultaeten/page.html /seitenbaum/kontakt/page.html /seitenbaum/impressum/page.html /seitenbaum/datenschutz/page.html /seitenbaum/hilfe-in-notfaellen/page.html /seitenbaum/studierende/fuer-studierende/page.html /seitenbaum/lehrende-und-beschaeftigte/fuer-lehrende-und-beschaeftigte/page.html /seitenbaum/home/page.html /seitenbaum/home/allgemeinwissenschaftliche-wahl-wahlpflichtfaecher/page.html /seitenbaum/home/bibliothek/page.html /seitenbaum/home/career-service/page.html /seitenbaum/home/contacting/page.html /seitenbaum/home/hochschulkommunikation/page.html /seitenbaum/home/hochschulleitung/page.html /institutionen/hochschulservice-fuer-familie-gleichstellung-und-gesundheit/page.html /seitenbaum/hochschule/oeffnungszeiten-der-hochschule/page.html /seitenbaum/home/qualitaetsmanagement/page.html /seitenbaum/home/lehr-und-kompetenzentwicklung/page.html /institutionen/zentrale-studienberatung/page.html /seitenbaum/hom

Nun löschen wir alle Links, die kein Inhalt haben (leere Links). 

In [97]:
def remove_empty_links(links):
    cleaned_links = [link for link in links if link.strip()]
    return cleaned_links

In [198]:
df = pd.DataFrame({"link": remove_empty_links(df["link"])})
print(*df["link"])
print(len(df["link"]))

/seitenbaum/home/page.html /seitenbaum/hochschule/page.html /seitenbaum/fakultaeten/page.html /seitenbaum/kontakt/page.html /seitenbaum/impressum/page.html /seitenbaum/datenschutz/page.html /seitenbaum/hilfe-in-notfaellen/page.html /seitenbaum/studierende/fuer-studierende/page.html /seitenbaum/lehrende-und-beschaeftigte/fuer-lehrende-und-beschaeftigte/page.html /seitenbaum/home/page.html /seitenbaum/home/allgemeinwissenschaftliche-wahl-wahlpflichtfaecher/page.html /seitenbaum/home/bibliothek/page.html /seitenbaum/home/career-service/page.html /seitenbaum/home/contacting/page.html /seitenbaum/home/hochschulkommunikation/page.html /seitenbaum/home/hochschulleitung/page.html /institutionen/hochschulservice-fuer-familie-gleichstellung-und-gesundheit/page.html /seitenbaum/hochschule/oeffnungszeiten-der-hochschule/page.html /seitenbaum/home/qualitaetsmanagement/page.html /seitenbaum/home/lehr-und-kompetenzentwicklung/page.html /institutionen/zentrale-studienberatung/page.html /seitenbaum/hom

Man sieht, dass wir 0 Links ausgefiltert haben, wenn man die Anzahl der Links vor und nach der Filterung vergleicht.

### Externe Links finden- nur zur Visualisierung (optionale Funktion)

Jetzt können wir Mal nachschauen, auf welche externen Seiten die Startseite des THN-Intranet verweist.

In [99]:
def find_extern_urls(urls):
    external_links = []
    for link in urls:
        if link.startswith("http"):
            external_links.append(link)

    return external_links

In [199]:
external_links = find_extern_urls(df["link"])
print("Anzahl externer Links: ", len(external_links))
print(external_links)

Anzahl externer Links:  3
['https://elearning.ohmportal.de/', 'http://jobboerse.th-nuernberg.de/', 'http://ohm.kh-netzwerk.de/forum/rund-um-den-studienalltag']


Die Seite verweist also auf elearning, auf jobboerse und ein Forum.

### Interne Links filtern

Wir filtern nun noch alle Links heraus, die keine HTML-inhalte besitzen, wie z.B. pdf oder xml Dateien, die mail Links enthalten oder die nicht auf die THN Webseite referieren.

In [145]:
def filter_intern_links(urls):
    filtered_links = []
    for url in urls:
        if url.startswith("http"):
            continue
        elif url.startswith("mailto:"):
            continue
        elif url.startswith("javascript:"):
            continue
        elif url.startswith("&#"): # is encoded mailto
            continue
        elif ".xml" in url:
            continue
        elif ".docx" in url:
            continue
        elif ".pdf" in url:
            continue
        elif url == "/":
            continue
        elif url == "&":
            continue
        else:
            filtered_links.append(url)
    return filtered_links

In [200]:
intern_links = filter_intern_links(df["link"])
print("Anzahl interner Links: ", len(intern_links))
print(intern_links)

Anzahl interner Links:  55
['/seitenbaum/home/page.html', '/seitenbaum/hochschule/page.html', '/seitenbaum/fakultaeten/page.html', '/seitenbaum/kontakt/page.html', '/seitenbaum/impressum/page.html', '/seitenbaum/datenschutz/page.html', '/seitenbaum/hilfe-in-notfaellen/page.html', '/seitenbaum/studierende/fuer-studierende/page.html', '/seitenbaum/lehrende-und-beschaeftigte/fuer-lehrende-und-beschaeftigte/page.html', '/seitenbaum/home/page.html', '/seitenbaum/home/allgemeinwissenschaftliche-wahl-wahlpflichtfaecher/page.html', '/seitenbaum/home/bibliothek/page.html', '/seitenbaum/home/career-service/page.html', '/seitenbaum/home/contacting/page.html', '/seitenbaum/home/hochschulkommunikation/page.html', '/seitenbaum/home/hochschulleitung/page.html', '/institutionen/hochschulservice-fuer-familie-gleichstellung-und-gesundheit/page.html', '/seitenbaum/hochschule/oeffnungszeiten-der-hochschule/page.html', '/seitenbaum/home/qualitaetsmanagement/page.html', '/seitenbaum/home/lehr-und-kompetenze

Nun haben wir 4 Links entfernt.

### Dublikate entfernen und sortieren

Jetzt können wir die duplikate entfernen und anschließend alphabetisch sortieren.

In [194]:
def sort_and_remove_dublicates(df):
    if 'html' in df.columns:
        df = df.sort_values(by='html', ascending=False)
        df = df.drop_duplicates(subset='link', keep='first')
        df = df.reset_index(drop=True)
    else:
        df = df.sort_values(by='link', ascending=False)
        df = df.drop_duplicates(subset='link', keep='first')
        df = df.reset_index(drop=True)
    return df

In [201]:
intern_links = filter_intern_links(df["link"])
df = pd.DataFrame({"link": intern_links})
df = sort_and_remove_dublicates(df)
print("Anzahl interner Links (ohne Dublikate): ", len(df["link"]))
print(*df["link"])

Anzahl interner Links (ohne Dublikate):  41
/seitenbaum/studierende/rund-ums-studium/servicefinder/page.html /seitenbaum/studierende/mein-studium/online-services/page.html /seitenbaum/studierende/mein-studium/download-center/antraege-und-meldungen/page.html /seitenbaum/studierende/fuer-studierende/page.html /seitenbaum/lehrende-und-beschaeftigte/fuer-lehrende-und-beschaeftigte/page.html /seitenbaum/kontakt/page.html /seitenbaum/impressum/page.html /seitenbaum/home/vermietungen/page.html /seitenbaum/home/technik-und-facility-management/page.html /seitenbaum/home/studienfinanzierung/page.html /seitenbaum/home/studienbuero/studierendenservicestudienbuero/page.html /seitenbaum/home/qualitaetsmanagement/page.html /seitenbaum/home/page.html /seitenbaum/home/lehr-und-kompetenzentwicklung/page.html /seitenbaum/home/hochschulleitung/page.html /seitenbaum/home/hochschulkommunikation/page.html /seitenbaum/home/contacting/page.html /seitenbaum/home/career-service/page.html /seitenbaum/home/bibliot

Anhand der Anzahl der Links vor und nach der Duplikaten Entfernung, sieht man das 14 doppelt vorkommende Links entfernet werden.

### Links abspeichern

Für unsere weiteren Schritte werden wir immer nur interne Links verwenden, deshalb speichern wir an dieser Stelle mal die internen Links ab.

In [153]:
db_save_df(df, "only_links")

### Downloaden der files

Jetzt können wir mit dem downloaden anfangen.

Diese Funktion lädt nun alle Html files zu den Links herunter und speichert sie im Dataframe neben den "link" in einer Spalte "html".

In [106]:
def download_all_urls(links):
    htmls = []
    for link in tqdm(links):
        url = "https://intern.ohmportal.de/" + link
        html = download_html_from_url(url)
        htmls.append(html)
    return htmls

In [154]:
df["html"]=download_all_urls(df["link"])

  2%|▏         | 1/41 [00:00<00:16,  2.41it/s]

Umleitung zu externer Seite verhindert. URL: https://intern.ohmportal.de/seitenbaum/home/page.html


 20%|█▉        | 8/41 [00:02<00:08,  3.88it/s]

Umleitung zu externer Seite verhindert. URL: https://intern.ohmportal.de/institutionen/fakultaeten/elektrotechnik-feinwerktechnik-informationstechnik/elektrotechnik-feinwerktechnik-informationstechnik/page.html


 22%|██▏       | 9/41 [00:02<00:09,  3.48it/s]

...

Umleitung zu externer Seite verhindert. URL: https://www.th-nuernberg.de/impressum/


 88%|████████▊ | 36/41 [00:18<00:06,  1.25s/it]

Umleitung zu externer Seite verhindert. URL: https://www.th-nuernberg.de/wie-erreichen-sie-uns/


100%|██████████| 41/41 [00:19<00:00,  2.05it/s]

Umleitung zu externer Seite verhindert. URL: https://www.th-nuernberg.de/unsere-beratungs-und-serviceangebote-fuer-sie/


In [115]:
def remove_rows_with_empty_html(df):
    df = df[df["html"] != ""]  
    df.reset_index(drop=True, inplace=True)
    return df


In [155]:
df = remove_rows_with_empty_html(df)

In [156]:
print(len(df))

29


Nun sind 12 Reihen entfernt, die keinen HTML Inhalt hatten, da sie zum Beispiel auf eine andere Domäne verweisen wurden.

Wir können die Daten an dieser Stelle abspeichern.

In [157]:
db_save_df(df, "intranet_html_iter_01")

### Weitere Iterationsstufen

Wenn wir ab diesem Abschnitt starten können wir die vorher gesammelten Daten neu laden.

In [202]:
df = db_get_df("intranet_html_iter_01")
print(len(df["link"]))

29


In [203]:
non_none_html_rows = df[df["html"].notnull()]  # Filtere die Zeilen, in denen "html" nicht "None" ist
print(len(non_none_html_rows))

29


Jetzt können wir die heruntergeladenen HTML files nach weiteren Links durchsuchen und Sie dem Dataframe hinzufügen

In [131]:
def find_all_links_in_html(htmls):
    all_links = []
    for html in tqdm(htmls):
        links = get_links_from_one_html(html)
        links= clean_links(links)
        links= remove_empty_links(links)
        links= filter_intern_links(links)
        [all_links.append(link) for link in links]
    return all_links

In [204]:
all_links = find_all_links_in_html(df["html"])
len(all_links)

100%|██████████| 29/29 [00:00<00:00, 134.88it/s]


979

Wir haben jetzt also 979 Links gesammelt, davon sind aber viele Dublikate.

Nun führen wir die neu gesammelten Links mit den ursprünglichen Links zusammen, wobei die neuen Links ein "None" Wert für die "html" Spalte bekommen.

In [205]:
df_new = pd.DataFrame({"link": all_links, "html": None})
df = pd.concat([df, df_new])

In [206]:
df = sort_and_remove_dublicates(df)
len(df["link"])

285

Gefiltert nach dublikaten haben wir nun also noch 285 Links

In [207]:
print(*df["link"])

/seitenbaum/hochschule/oeffnungszeiten-der-hochschule/page.html /institutionen/zentrale-studienberatung/page.html /institutionen/zentrale-it/page.html /institutionen/fakultaeten/werkstofftechnik/page.html /seitenbaum/home/vermietungen/page.html /seitenbaum/home/technik-und-facility-management/page.html /seitenbaum/home/studienbuero/studierendenservicestudienbuero/page.html /seitenbaum/home/qualitaetsmanagement/page.html /seitenbaum/studierende/mein-studium/online-services/page.html /institutionen/fakultaeten/maschinenbau-und-versorgungstechnik/page.html /seitenbaum/home/lehr-und-kompetenzentwicklung/page.html /seitenbaum/home/page.html /institutionen/hochschulservice-fuer-familie-gleichstellung-und-gesundheit/page.html /seitenbaum/home/hochschulleitung/page.html /seitenbaum/home/hochschulkommunikation/page.html /seitenbaum/hochschule/page.html /seitenbaum/studierende/fuer-studierende/page.html /seitenbaum/lehrende-und-beschaeftigte/fuer-lehrende-und-beschaeftigte/page.html /seitenbaum/

In [141]:
def print_unique_link_endings(df):
    endings = set() 
    
    for link in df["link"]:
        parts = link.split("/") 
        if len(parts) > 0:
            ending = parts[-1]  
            endings.add(ending)
    
    for ending in endings:
        print(ending)

In [177]:
print_unique_link_endings(df)

page.html
index.php


In [208]:
non_none_html_rows = df[df["html"].notnull()]  # Filtere die Zeilen, in denen "html" nicht "None" ist
print(len(non_none_html_rows))

29


### Iteratives Downloaden

Um den Daten nun weitere Webseiten hinzuzufügen, können wir für jede weitere URL schauen, ob sie schon heruntergeladen wurde. Wenn nicht, dann laden wir sie jetzt herunter.

In [209]:
def update_df_with_html(df):
    for index, row in tqdm(df.iterrows()):
        if pd.isna(row['html']) or row['html'] == '':
            url = "https://intern.ohmportal.de/" + row["link"]
            html = download_html_from_url(url)
            df.at[index, 'html'] = html
    
    return df

In [166]:
print(df.loc[2])


link    /institutionen/fakultaeten/angewandte-chemie/f...
html                                                 None
Name: 2, dtype: object


In [210]:
df = update_df_with_html(df)

30it [00:00, 35.88it/s]

Umleitung zu externer Seite verhindert. URL: https://www.th-nuernberg.de/wie-erreichen-sie-uns/
Umleitung zu externer Seite verhindert. URL: https://www.th-nuernberg.de/impressum/
Umleitung zu externer Seite verhindert. URL: https://www.th-nuernberg.de/datenschutz/


37it [00:06,  4.19it/s]

Umleitung zu externer Seite verhindert. URL: https://intern.ohmportal.de/seitenbaum/home/page.html


49it [00:17,  1.13it/s]

Umleitung zu externer Seite verhindert. URL: https://www.th-nuernberg.de/index.php?id=107


65it [00:29,  1.26it/s]

Umleitung zu externer Seite verhindert. URL: https://www.th-nuernberg.de/fakultaeten/bi/


66it [00:32,  2.04it/s]


SSLError: HTTPSConnectionPool(host='apps.bw.ohmportal.de', port=443): Max retries exceeded with url: /bwAktuell/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)')))

In [211]:
print(df["link"][66])

/institutionen/fakultaeten/betriebswirtschaft/online-services/page.html


### Iterationen


1. Iteration: Startseite TH-Intranet (~300 Links)
2. Iteration: Links von startseite aufrufen und HTML scrapen (~2000 Links)
3. Iteration: Links von diesen Seiten aufrufen und HTML scrapen (~ Links)

In [ ]:
def iteration(df):
    all_links = find_all_links_in_html(df["html"])
    df_new = pd.DataFrame({"link": all_links, "html": None})
    df = pd.concat([df, df_new])
    df = sort_and_remove_dublicates(df)
    df=update_df_with_html(df)
    return df

In [ ]:
df=iteration(df)

### Texte extrahieren

Als nächstes müssen wir aus den rohen HTML Dokumenten die unrelevanten Daten aussortieren

In [ ]:
df = db_get_df("html_iter_01")

Die nachfolgende Funktion bestimmt, ob ein Beautifulsoup geparstes HTML Element sichtbar ist oder nicht.

In [ ]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

Jetzt bestimmen wir eine Funktion, die ein HTML als Input bekommt und daraus die Texte und Titeln bestimmt.

Die HTML Seiten haben 3 verschiedene Strukturen:

- "main" (https://www.th-nuernberg.de/hochschule-region/organisation-und-struktur/hochschulleitung-und-gremien/)
- "div", {'class': 'portal'} (https://www.th-nuernberg.de/fakultaeten/bi/)
- "div", {'class': 'page-wrap'} (https://www.th-nuernberg.de/studium-karriere/studien-und-bildungsangebot/duale-studienmodelle/studium-mit-vertiefter-praxis/)

In [ ]:
def get_content(file):
    soup = BeautifulSoup(file,"lxml")

    title = soup.find("title")
    if title:
        title = title.text
    else:
        title = ""

    main = soup.find("main")
    portal = soup.find("div", {'class': 'portal'})
    page_container = soup.find("div", {'class': 'page-wrap'})
    
    visible_texts = ""
    if main:
        container = main.find("div" ,{'class': 'container'}, recursive=False)
        if container:
            texts = container.find_all(text=True)
            visible_texts = filter(tag_visible, texts)

    elif portal:
        texts = portal.find_all(text=True)
        visible_texts = filter(tag_visible, texts)

    elif page_container:
        container = page_container.find("div" ,{'class': 'container'}, recursive=False)
        if container:
            texts = container.find_all(text=True)
            visible_texts = filter(tag_visible, texts)

    return {
        "title":    title,
        "text":     u" ".join(t.strip() for t in visible_texts)
    }

Nun speichern wir die Texte sowie die dazugehörigen Titeln in den Dataframe.

In [ ]:
parsed_texts = []
titles = []
for html in tqdm(df["html"]):
    content = get_content(html)
    parsed_texts.append(content["text"])
    titles.append(content["title"])

df["text"] = parsed_texts
df["title"] = titles

In [ ]:

print(df[df["text"] != ""]["url"])

In [ ]:
db_save_df(df, "html_attribute")